In [103]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import requests
import time
import os

In [107]:
import json
f = open('./token.json','r')
access = json.load(f)
f.close()

In [108]:
class idealista_search_api():
  
  def __init__(self,access:dict):
    self.token = access['access_token']
    self.token_type = access['token_type']
    self.country = 'es'
    self.api_version = '3.5'
    self.endpoint = f'https://api.idealista.com/{self.api_version}/{self.country}/search'
    
  def request(self,payload:dict):
    headers = {
        'Authorization': f'{self.token_type} {self.token}'
    }
    response = requests.post(self.endpoint,params=payload,headers=headers)
    if response.status_code == 200:
      return response
    response.raise_for_status()

  def extract_data(self,payload:dict) -> pd.DataFrame:
    print('this is the payload:',payload)
    data = json.loads(self.request(payload).text)
    totalPages = data['totalPages']
    self.dataset = pd.DataFrame(data['elementList'])
    self.dataset.to_csv('idealista_real_estate_madrid_housing.csv')
    print('first extraction done')
    print(self.dataset)

    print(f'total pages: {totalPages}')

    for page in range(2,totalPages+1):
      time.sleep(1)
      print(page)
      payload['numPage'] = page
      data = json.loads(self.request(payload).text)
      self.dataset = self.dataset.append(data['elementList'])
      pd.DataFrame(data['elementList']).to_csv('idealista_real_estate_madrid_housing.csv',mode='a')

In [109]:
idealista = idealista_search_api(access)

In [ ]:
payload = {
    'operation':'sale',
    'propertyType':'homes',
    'maxItems':50,
    'order':'price',
    'sort':'desc',
    'locationId':'0-EU-ES-28-07-001-079'
    }

idealista.extract_data(payload)